In [11]:
import pandas as pd
import numpy as np
import csv 

## FUNCTIONAL CODE DONT REMOVE THIS
rateData = pd.read_json('episoderatings.json',lines=True)
epCSV = pd.read_csv('seasons.csv')
impCSV = pd.read_csv('appearances_redo.csv')
impData = dict(impCSV)
epData = epCSV.to_json()

print(impData['sid'][100])

46


In [13]:
# Find top 25 imdb scores but initilize it with an empty array
finalTop100 = np.zeros((100,3),int)   
lenData = len(rateData['epno'])
for i in range(lenData):
    currentDemographic = rateData['demographic_counts'][i]
    currentScore = currentDemographic['IMDb users']

    # Need to add plus 1 to 
    currentData = np.array([rateData['epno'][i]+1,rateData['sid'][i],currentScore])

    # Check ths current data and see if the score is greater than any of the 
    for j in range(100):
        currentListing = finalTop100[j]

        # Case of the current score is greater than the current index then break the search in the for loop
        if currentListing[2] < currentScore: 
            finalTop100[j] = currentData
            break 

# Categorized decending order hi to lo :  EPISODE  / SEASON  /  RATING  
print('Top 100: \n')
print(finalTop100)

# Write the data to CSV 
with open("top100.csv", "wt") as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(["Episode #", "Season #", " IMDb Rating"])  # write header
    writer.writerows(finalTop100)

Top 100: 

[[  4  41 489]
 [ 10  39 413]
 [ 21  35 343]
 [  3  40 284]
 [  1  40 273]
 [ 16  38 273]
 [  1   1 265]
 [ 10  37 236]
 [ 22  36 230]
 [  1  39 212]
 [ 13  38 211]
 [  9  32 208]
 [  6  38 195]
 [  2  38 191]
 [ 21  34 173]
 [  2  37 170]
 [  2  35 170]
 [  7   1 169]
 [  2   1 158]
 [ 18  35 155]
 [  3  37 143]
 [  4  36 143]
 [ 16  35 139]
 [ 16  32 137]
 [ 20  21 137]
 [ 17  32 131]
 [  3   1 129]
 [  8  34 123]
 [  2  34 121]
 [  1  34 118]
 [  6  33 117]
 [ 13  31 110]
 [  5   1 109]
 [ 10  32  99]
 [  4  32  99]
 [  1  32  98]
 [  8   1  97]
 [  7  31  85]
 [ 18   3  84]
 [ 14   1  84]
 [  9   1  84]
 [ 11   1  80]
 [ 19   1  74]
 [ 16   1  74]
 [ 12   1  74]
 [ 15   1  69]
 [ 15   2  63]
 [ 23   1  63]
 [  6   2  61]
 [  5   2  61]
 [  1   2  61]
 [ 24   1  61]
 [ 22   1  61]
 [  4   2  56]
 [  2   2  55]
 [ 12   4  53]
 [  3   4  53]
 [ 10   3  53]
 [ 16   2  53]
 [ 17   2  51]
 [ 12   2  50]
 [ 18   2  49]
 [ 11   2  49]
 [ 12   3  46]
 [  4   3  46]
 [ 13   2  46]

In [24]:
# removing duplicated from list using collections.OrderedDict.fromkeys() 
from collections import OrderedDict 

# Extract the data relevant to the seasons that apear in top 100
relevantSeasons = finalTop100[:,1]
relevantSeasons = relevantSeasons.tolist()

# to remove duplicated from list 
reducedSeasons = list(OrderedDict.fromkeys(relevantSeasons)) 
reducedSeasons.sort()
print(reducedSeasons)

# Write the data to CSV 
with open("topRatedSeasons.csv", "wt") as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(["Seasons"])  # write header
    writer.writerow(reducedSeasons)

[1, 2, 3, 4, 5, 6, 8, 21, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]


In [25]:
# Find the casts of the seasons listed in reducedSeasons
castAppearances = np.array([['Member','Season #']])
for i in range(len(impData['aid'])):
    
    # Now loop the relevant seasons for a match 
    for j in range(len(reducedSeasons)): 
      
      if impData['sid'][i] == reducedSeasons[j]:
        castAppearances = np.append(castAppearances,[[impData['aid'][i],impData['sid'][i]]],axis = 0)

# Remove duplicates in 2D array
checkedVals = set() 
allAppearances = []
for item in castAppearances:
    currentVal = tuple(item)

    # Check value against complete set
    if currentVal not in checkedVals:
      allAppearances.append(currentVal)
      checkedVals.add(currentVal)

allAppearances = np.asanyarray(allAppearances)
allAppearances = np.delete(allAppearances,0, axis=0)
print(allAppearances)

# Write the data to CSV 
with open("seasonCasts.csv", "wt") as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(["Cast Member", "Season #"])  # write header
    writer.writerows(allAppearances)

[['Larry David' '41']
 ['Vanessa Bayer' '41']
 ['Beck Bennett' '41']
 ...
 ['Richard Belzer' '1']
 ['Janis Ian' '1']
 ['Valri Bromfield' '1']]


In [26]:
# Now extract all the roles used in the relevant seasons 

# Find the casts of the seasons listed in reducedSeasons
castRoleAppearances = np.array([['Member','Role','Season #']])
for i in range(len(impData['aid'])):
    
    # Now loop the relevant seasons for a match 
    for j in range(len(reducedSeasons)): 
      
      if impData['sid'][i] == reducedSeasons[j]:
        castRoleAppearances = np.append(castRoleAppearances,[[impData['aid'][i],impData['role'][i],impData['sid'][i]]],axis = 0)

# Remove duplicates in 2D array and also remove any duplicate values 
bannedVals = set()
bannedKeywords = ['publicist','dancer','anchor','sister','executive','boomer','custodian','soldier','announcer','professor','boyfriend','girlfriend','mother','producer','chef','teacher',
'cook','witness','father','bartender','kidnapper','grandma','girl','boy','voter','captain','employee','nurse','crew member','animal expert','wife','corporal','person','dancer','woman',
'ghost','husband','secretary','fan','doctor','sailor','player','narrator','ship builder','spectator','man','customer','spokesperson','user','student','instructor','daughter','son','guest',
'hostage','guy','officer','pledge','perfomer','brother','passenger','contestent','patron','priest','musician','student','friend','follower','double']

# Add banned vals to the set 
for val in bannedKeywords:
    bannedVals.add(val)

# Now sift through all data and sort accordingly
roleAppearances = []
for item in castRoleAppearances:
    tempVal = tuple(item)

    # Strange error where nan couldnt be detected so converted both things to strings and it solved the issue 
    strTempVal = str(tempVal[1])

    # Check value against complete banned role keyword set
    if (strTempVal not in bannedVals) and ('nan' != strTempVal):
      roleAppearances.append(tempVal)

# Need to clean up and remove the header of this 2D array, but also need to remove all 'nan' listings 
roleAppearances = np.asanyarray(roleAppearances)
roleAppearances = np.delete(roleAppearances,0,axis=0)
print(roleAppearances)

# Write the data to CSV 
with open("rolesAllSeasons.csv", "wt") as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(["Cast Member","Role Played","Season #"])  # write header
    writer.writerows(roleAppearances)

[['Larry David' 'Bernie Sanders' '41']
 ['Bill Corsair' 'pianist' '41']
 ['Darrell Hammond' 'Bill Clinton' '41']
 ...
 ['Garrett Morris' 'Harvey Morgomaster' '1']
 ['Gilda Radner' 'Mrs. Kromer' '1']
 ['Al Franken' 'caveman' '1']]


In [27]:
# Retrieve to number of times the a character/role and member combination appears, then just role 
countMembRole = np.array([['Member','Role','Appearances #']])
tempCompound = np.array(['temp'])
countRole = np.array([['Role','Appearances #']])
tempRole = np.array(roleAppearances[:,1])
# Remove duplicates in 2D array similar to before 
checkedPair = set() 
checkedRole = set() 
allPair = []
allRole = []

# Extract the necessary info from roleAppearances and begin extracting all unnique role counts
tempRole = np.delete(tempRole,0,axis=0)
reducedRoleList = list(set(tempRole))

# Find role counts
totalRoleCount = []
for role in reducedRoleList:
    roleCount = 0

    for i in range(len(tempRole)):
        if role == tempRole[i]:
            roleCount += 1
    totalRoleCount.append(roleCount)

# Constuct the final arrays 
for i in range(len(reducedRoleList)):
    countRole = np.append(countRole,[[reducedRoleList[i],totalRoleCount[i]]],axis=0)
countRole = np.asanyarray(countRole)
countRole = np.delete(countRole,0,axis=0)
print(countRole)

# Write the data to CSV 
with open("roleCount.csv", "wt") as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(["Role Played","Count #"])  # write header
    writer.writerows(roleAppearances)

[['crowd member' '3']
 ['Zack' '1']
 ['Paula Deen' '2']
 ...
 ['flight attendant' '5']
 ['roofer' '2']
 ['Benjamin Tellurin' '1']]


In [28]:
# Finally, find the inverse of the previous section. Find out how the most common passing role is (my guess is "guy")
indexCount = np.array([["Keyword","Count"]])
for i in range(len(bannedKeywords)):
    keywordCount = 0 
    # Check the current current band word and see how often it comes up 
    for j in range(len(castRoleAppearances)):
        if str(castRoleAppearances[j][1]) == str(bannedKeywords[i]):
            keywordCount += 1

    indexCount = np.append(indexCount,[[bannedKeywords[i],keywordCount]],axis=0)


# Always stored as KEYWORD  /  COUNT # 
bannedKeywordCount = np.asanyarray(indexCount)
bannedKeywordCount = np.delete(bannedKeywordCount,0,axis=0)

print(np.array(bannedKeywordCount))

# Write the data to CSV 
with open("blockedRoleCount.csv", "wt") as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(["Non-Counted Role Played","Count #"])  # write header
    writer.writerows(roleAppearances)

[['publicist' '0']
 ['dancer' '101']
 ['anchor' '7']
 ['sister' '11']
 ['executive' '61']
 ['boomer' '0']
 ['custodian' '0']
 ['soldier' '35']
 ['announcer' '1035']
 ['professor' '17']
 ['boyfriend' '17']
 ['girlfriend' '12']
 ['mother' '125']
 ['producer' '6']
 ['chef' '8']
 ['teacher' '51']
 ['cook' '17']
 ['witness' '4']
 ['father' '89']
 ['bartender' '30']
 ['kidnapper' '0']
 ['grandma' '9']
 ['girl' '286']
 ['boy' '3']
 ['voter' '11']
 ['captain' '14']
 ['employee' '102']
 ['nurse' '44']
 ['crew member' '25']
 ['animal expert' '0']
 ['wife' '108']
 ['corporal' '1']
 ['person' '0']
 ['dancer' '101']
 ['woman' '82']
 ['ghost' '1']
 ['husband' '91']
 ['secretary' '23']
 ['fan' '17']
 ['doctor' '43']
 ['sailor' '11']
 ['player' '63']
 ['narrator' '57']
 ['ship builder' '0']
 ['spectator' '19']
 ['man' '23']
 ['customer' '154']
 ['spokesperson' '104']
 ['user' '25']
 ['student' '204']
 ['instructor' '12']
 ['daughter' '46']
 ['son' '33']
 ['guest' '204']
 ['hostage' '7']
 ['guy' '438']